In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import matplotlib.pyplot as plt
import seaborn as sns

import json
import gc

from scipy.stats import uniform

import warnings
warnings.filterwarnings("ignore")

from sklearn.svm import SVR, LinearSVR
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# The device, geonetwork, totals, trafficSource columns have multiple values within 
# their corresponding columns
# Define the load_df function to substract these values
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    # Use json_normalize function to clean these columns which have the embedding structure
    for column in JSON_COLUMNS:
        column_as_df = pd.json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
df = load_df('/kaggle/input/ga-customer-revenue-prediction/train.csv')

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# The df.info shows that for columns in [totals.bounces, totlas, newVisits, transaction Revenue, trafficSource.keyword, trafficSource.isTrueDirect, trafficSource.referralPath,  50  trafficSource.adwordsClickInfo.gclId               21561 non-null   objtrafficSource.adwordsClickInfo.adNetworkType
# trafficSource.adwordsClickInfo.isVideoAd, trafficSource.adContent, trafficSource.campaignCode]
# There are NULL values.
# First check columns contain nulls with their null percentage
# Then CLean up the data
null_percentage = pd.DataFrame()
for col in df.columns:
    if df[col].isnull().sum() > 0:
        null_percentage.loc[col,'NullPercentage'] = (df[col].isnull().sum())/len(df) * 100 
print(null_percentage)

Note that there is 98.9% null percentage in our target column 'totals.transactionRevenue', while it is normal, meaning that most of the customers didn't make an trascation.
'trafficSource.adContent' and 'trafficSource.campaignCode' contains too much NULL value, thus we consider to drop them.
'totals.newVisits' & 'totals.pageviews' could be imputed and filled by mode.

In [ ]:
def clean_up(df):
    df.drop('trafficSource.adContent', axis=1, inplace=True)
    df.drop('trafficSource.campaignCode', axis=1, inplace=True)
    df["totals.transactionRevenue"] = df['totals.transactionRevenue'].fillna("0").astype(float)
    
    df["totals.pageviews"] = df["totals.pageviews"].fillna(0).astype(int)
    df["totals.newVisits"] = df["totals.newVisits"].fillna(0).astype(int)
    return df

df = clean_up(df)

In [ ]:
df.describe()

In [ ]:
df.head(5)

In [ ]:
# Select the top 10 countries with most transcation records
top_countries = df[["geoNetwork.country"]].groupby("geoNetwork.country").size().sort_values(ascending=False)[:10].index.to_numpy()
avg_totals_transactionRevenue = df.groupby('geoNetwork.country')['totals.transactionRevenue'].mean().reset_index()
df_top_countries = avg_totals_transactionRevenue[avg_totals_transactionRevenue["geoNetwork.country"].isin(top_countries)]
df_top_countries

plt.figure(figsize=(18,4))
sns.barplot(data=df_top_countries, x="geoNetwork.country", y="totals.transactionRevenue")
plt.show()

check the impact of geoNetwork.country, geoNetwork.region, geoNetwork.metro and geoNetwork.city

In [ ]:
# Select the top 10 cities with most transcation records
top_cities = df[["geoNetwork.city"]].groupby("geoNetwork.city").size().sort_values(ascending=False)[:10].index.to_numpy()
avg_totals_transactionRevenue = df.groupby('geoNetwork.city')['totals.transactionRevenue'].mean().reset_index()
df_top_cities = avg_totals_transactionRevenue[avg_totals_transactionRevenue["geoNetwork.city"].isin(top_cities)]
df_top_cities

plt.figure(figsize=(18,4))
sns.barplot(data=df_top_cities, x="geoNetwork.city", y="totals.transactionRevenue")
plt.show()

In [ ]:
# Select the top 10 regions with most transcation records
top_regions = df[["geoNetwork.region"]].groupby("geoNetwork.region").size().sort_values(ascending=False)[:10].index.to_numpy()
avg_totals_transactionRevenue = df.groupby('geoNetwork.region')['totals.transactionRevenue'].mean().reset_index()
df_top_regions = avg_totals_transactionRevenue[avg_totals_transactionRevenue["geoNetwork.region"].isin(top_regions)]
df_top_regions

plt.figure(figsize=(18,4))
sns.barplot(data=df_top_regions, x="geoNetwork.region", y="totals.transactionRevenue")
plt.show()